In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB0
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import os
import numpy as np

# Define paths (replace with your dataset path)
base_dir = "/content/dataset"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")

# Image dimensions & parameters
img_size = (224, 224)
batch_size = 32
num_classes = 4

def create_data_generators():
    train_datagen = ImageDataGenerator(
        rescale=1.0/255,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
    test_datagen = ImageDataGenerator(rescale=1.0/255)
    
    train_generator = train_datagen.flow_from_directory(
        train_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical'
    )
    test_generator = test_datagen.flow_from_directory(
        test_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical'
    )
    
    return train_generator, test_generator

def build_model(base_model):
    base_model.trainable = False  # Freeze the base model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=x)
    return model

def compile_and_train(model, train_generator, test_generator, model_name):
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    history = model.fit(train_generator, epochs=10, validation_data=test_generator)
    model.save(f"{model_name}.h5")

def train_vgg():
    print("Training VGG16 Model...")
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    model = build_model(base_model)
    train_generator, test_generator = create_data_generators()
    compile_and_train(model, train_generator, test_generator, "VGG16_Tree_Classification")

def train_resnet():
    print("Training ResNet50 Model...")
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    model = build_model(base_model)
    train_generator, test_generator = create_data_generators()
    compile_and_train(model, train_generator, test_generator, "ResNet50_Tree_Classification")

def train_efficientnet():
    print("Training EfficientNetB0 Model...")
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    model = build_model(base_model)
    train_generator, test_generator = create_data_generators()
    compile_and_train(model, train_generator, test_generator, "EfficientNet_Tree_Classification")

def evaluate_models():
    test_datagen = ImageDataGenerator(rescale=1.0/255)
    test_generator = test_datagen.flow_from_directory(
        test_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical', shuffle=False
    )
    
    models = {
        "VGG16": "VGG16_Tree_Classification.h5",
        "ResNet50": "ResNet50_Tree_Classification.h5",
        "EfficientNetB0": "EfficientNet_Tree_Classification.h5"
    }
    
    for name, model_path in models.items():
        print(f"Evaluating {name}...")
        model = load_model(model_path)
        loss, accuracy = model.evaluate(test_generator)
        print(f"{name} - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

# Uncomment the model you want to train
# train_vgg()
# train_resnet()
# train_efficientnet()

# Uncomment to evaluate models
# evaluate_models()
